# Сравнение качества морфологических теггеров

*Прошу прощения за просрочку даже делайна +2 дня*

**Задание**

1. (1 балл) Создание, разметка корпуса и объяснение того, почему этот текст подходит для оценки (какие моменты вы тут считаете трудными для автоматического посттеггинга и почему, в этом вам может помочь второй ридинг). Не забывайте, что разные теггеры могут использовать разные тегсеты: напишите комментарий о том, какой тегсет вы берёте для разметки и почему.
2. (3 балла) Потом вам будет нужно взять три POS теггера для русского языка (udpipe, stanza, natasha, pymorphy, mystem, spacy, deeppavlov) и «прогнать» текст через каждый из них.
3. (2 балла) Затем оценим accuracy для каждого теггера. Заметьте, что в разных системах имена тегов и части речи могут отличаться, – вам надо будет свести это всё к единому стандарту с помощью какой-то функции-конвертера и сравнить с вашим размеченным руками эталоном - тоже с помощью какого-то кода или функции.
4. (2 балла) Дальше вам нужно взять лучший теггер для русского языка и с его помощью написать функцию (chunker), которая выделяет из размеченного текста 3 типа n-грамм, соответствующих какому-то шаблону (к примеру не + какая-то часть речи или NP или сущ.+ наречие и тд)
5. (2-3 балла) В предыдущем дз многие из вас справедливо заметили, что если бы мы могли класть в словарь не только отдельные слова, но и словосочетания, то программа работала бы лучше. Предложите 3 шаблона (слово + POS-тег / POS-тег + POS-тег) запись которых в словарь, по вашему мнению, улучшила бы качество работы программы из предыдущей домашки. Балл за объяснение того, почему именно эти группы вы взяли, балл за встраивание функции в программу из предыдущей домашки, балл за сравнение качества предсказания тональности с улучшением и без (это бонусный одиннадцатый балл).

## 1. Описание данных

В качестве корпуса был взят фрагмент из сказки Сергея Козлова "Трям! Здравствуйте!". 

Трудные моменты для автоматического постеггинга:
- Новые слова: например, "Тили-мили-трямдия". Скорее всего в словаре теггеров нет такого слова, а значит, чтобы установить его часть речи необходимо прибегнуть к subword segmentation, что делают не все теггеры
- Слова с дефисами: тот же пример, если перед постеггингом токенизатор разобьет такое слово на три, это будет неверно.
- Неоднозначная нотация некоторых частей речи: например, порядковые числительные -- прилагательное или числительное? Зависит от решения создателей постеггинга. Имена персонажей -- сущ или имя собственное?

Для разметки я выбрала тегсет UD, поскольку я писала на втором курсе курсач по синтагрусу в разметке UD, и просто лучше других её знаю. А ещё тегсет UD менее подробный, чем, скажем, pymorphy (OpenCorpora), поэтому свести второе к первому более-менее однозначно можно, а вот первое ко второму уже много сложнее, надо контекстные правила подключать.

**Выбранные POS теггеры и объяснения, почему они**
- stanza -- использует корпуса UD, но гораздо проще работать через питон
- pymorphy -- тоже легко работается через питон, работает на другом корпусе (OpenCorpora), интересно сравнить
- natasha -- разметка в формате UD, но принцип работы совсем другой, на вручную составленных правилах, хотя использует внутри себя pymorphy (не очень понимаю, как), интересно сравнить

In [123]:
with open("text_pure.txt", encoding='utf-8') as f:
    text = f.read()

## 2. POS теггинг

### stanza

In [124]:
import stanza
stanza.download('ru')

nlp = stanza.Pipeline(lang='ru', processors='tokenize,pos')

2021-10-06 02:58:03 INFO: Downloading default packages for language: ru (Russian)...
2021-10-06 02:58:04 INFO: File exists: /Users/tasia/stanza_resources/ru/default.zip.
2021-10-06 02:58:08 INFO: Finished downloading models and saved to /Users/tasia/stanza_resources.
2021-10-06 02:58:08 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2021-10-06 02:58:08 INFO: Use device: cpu
2021-10-06 02:58:08 INFO: Loading: tokenize
2021-10-06 02:58:08 INFO: Loading: pos
2021-10-06 02:58:08 INFO: Done loading processors!


In [125]:
doc = nlp(text)

tags_stanza = []
for sent in doc.sentences:
    for word in sent.words:
        tags_stanza.append([word.text, word.upos])

### pymorphy

In [126]:
from nltk import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [127]:
# OC tag: UD tag
converter = {
    'NOUN': 'NOUN', # не различает имена собственные
    'ADJF': 'ADJ',
    'ADJS': 'ADJ',
    'COMP': 'ADV',
    'VERB': 'VERB', # не различает вспомогательные глаголы
    'INFN': 'VERB',
    'PRTF': 'VERB', # сознательно будем ошибаться везде, где причастие в атрибутивной функции
    'PRTS': 'VERB',
    'GRND': 'VERB',
    'NUMR': 'NUM',
    'ADVB': 'ADV',
    'NPRO': 'PRON', # не различает DET
    'PRED': 'ADV', # ?
    'PREP': 'ADP',
    'CONJ': 'CCONJ', # не различает сочинительные и подчинительные союзы
    'PRCL': 'PART',
    'INTJ': 'INTJ',
    None: 'X'
    # нет PUNCT!
}

In [128]:
def opencorpora_to_ud_ru(oc_pos_tag):
    return converter[oc_pos_tag]

In [129]:
# Тут всё будет плохо, тк очень топорно токенизирую + беру только самый вероятный разбор
text_token = word_tokenize(text)
tags_pymorphy = [[word, opencorpora_to_ud_ru(morph.parse(word)[0].tag.POS)] for word in text_token]

### natasha

In [143]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,

    Doc
)

In [152]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [153]:
doc = Doc(text)

doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [154]:
tags_natasha = [tuple([word.text, word.pos, (word.start, word.stop)]) for word in doc.tokens]

## 3. Оценка accuracy

In [133]:
with open("text_gold.txt", encoding='utf-8') as f:
    tags_gold = [line.split() for line in f if '\t' in line]

In [134]:
def add_index(tags):
    start = 0
    for i, word_info in enumerate(tags):
        word = word_info[0]
        start = text.index(word, start)
        end = start + len(word)
        word_info.append((start, end))
        tags[i] = tuple(word_info)
        start = end
    return tags

In [135]:
tags_gold = add_index(tags_gold)
tags_stanza = add_index(tags_stanza)
tags_pymorphy = add_index(tags_pymorphy)

In [155]:
pos_taggers = {'stanza': tags_stanza, 'pymorphy': tags_pymorphy, 'natasha': tags_natasha}

gold = set(tags_gold)
num_words = len(gold)
for name, tags in pos_taggers.items():
    accuracy = len(gold & set(tags)) / num_words
    print(f"Accuracy of {name}: {accuracy}")

Accuracy of stanza: 0.8850574712643678
Accuracy of pymorphy: 0.42758620689655175
Accuracy of natasha: 0.9103448275862069


Лучший POS теггер: **natasha**

## 4. Chunker

In [156]:
n_grams = [('PART', 'VERB'), ('ADV', 'VERB'), ('ADJ', 'NOUN')]

In [ ]:
# поскольку уже запускала наташу выше, это можно не повторять

# from natasha import (
#     Segmenter,
#     MorphVocab,
    
#     NewsEmbedding,
#     NewsMorphTagger,

#     Doc
# )

# segmenter = Segmenter()
# morph_vocab = MorphVocab()

# emb = NewsEmbedding()
# morph_tagger = NewsMorphTagger(emb)

In [166]:
def chunker(just_text, n_gram):
    chunks = []
    
    doc = Doc(just_text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    
    for i in range(len(n_gram) - 1, len(doc.tokens)):
        n = i - len(n_gram) + 1
        words_pos = tuple([doc.tokens[n].pos, doc.tokens[i].pos])
        if words_pos == n_gram:
            chunks.append(doc.tokens[n].text + " " + doc.tokens[i].text)
    return chunks

In [172]:
text = input("Введите текст, из которого вы желаете извлечь n-граммы типа выше: ")
for n_gram in n_grams:
    print(n_gram, chunker(text, n_gram))

Введите текст, из которого вы желаете извлечь n-граммы типа выше: Я так сильно люблю Санкт-Петербург. И не люблю Москву. Это очень шумный город. А ещё в Питере есть красивое море.
('PART', 'VERB') ['не люблю']
('ADV', 'VERB') ['сильно люблю']
('ADJ', 'NOUN') ['шумный город', 'красивое море']


## 5. Улучшения к дз1

Шаблоны для улучшения работы анализатора тональности (общая идея -- провести минимальный поверхностный синтаксис для установления связей с помощью синтаксических шаблонов):
- PART + VERB -- так ловится большинство отрицаний, "не понравился", "не рекомендую" и т.д.
- ADV + VERB -- так ловятся глаголы и какие-то тонально окрашенные наречия, "ужасно отстирывает", "очень понравился" и т.д.
- ADJ + NOUN -- так ловятся кусочки именных группы с прилагательным, "хороший продукт", "плохой кондиционер" и т.д.